In [1]:
options = ["piedra", "tijeras", "papel"]

In [2]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
        
    return result

In [3]:
search_winner("papel", "tijeras")

2

In [4]:
test = [
    ["piedra", "piedra", 0],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [5]:
from random import choice
def get_choice():
    return choice(options)

In [6]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: papel player2: tijeras Winner: 2 
player1: piedra player2: tijeras Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: tijeras player2: tijeras Winner: 0 
player1: papel player2: tijeras Winner: 2 
player1: tijeras player2: papel Winner: 1 
player1: piedra player2: piedra Winner: 0 
player1: piedra player2: piedra Winner: 0 


In [7]:
def str_to_list(option):
    if option=="piedra":
        res = [1,0,0]
    elif option=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"])) #La función map() toma una función y una lista y aplica esa función a cada elemento de esa lista, produciendo una nueva lista. 
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [8]:
from sklearn.neural_network import MLPClassifier

In [9]:
clf = MLPClassifier(verbose=False, warm_start=True) # False para que no saque tantos datos al usarlo
# True para datos de entrenamiento 

In [10]:
model = clf.fit([data_X[0]], [data_y[0]]) # papel gana piedra
                #piedra    , papel
print(model)

MLPClassifier(warm_start=True)


C:\Users\angel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0} # Para guardar estadisticas
    
    data_X = [] # Listas vacias
    data_y = []
    
    for i in range(iters): 
        player1 = get_choice() # player 1 tiene una opcion

        predict = model.predict_proba([str_to_list(player1)])[0] # decide que hacer
        # predice que le diras al jugador 
        
        if predict[0] >= 0.95: # si esta 95 % seguro de que es la opcion 0
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice() # si ninguna se cumple elige random
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2) # devolvera 0 si es empate, 1 si gana el 1, 2 si gana el 2        
        if debug==True:   
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:# Si gana player 2 lo entrenamos
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [12]:
score, data_X, data_y = play_and_learn(1, debug=True) # el score se va al score, data_x al data_x, etc
print(data_X) 
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"])))) # porcentaje que ha ganado
if len(data_X): # si tenemos respuesta si gano debe aprender
    model = model.partial_fit(data_X, data_y) #Vamos a rentrenar

Player1: piedra Player2 (modelo): [0.03670797 0.02513857 0.95066718] --> papel
Comprobamos: p1 VS p2: 2
[[1, 0, 0]]
[[0, 0, 1]]
Score: {'win': 1, 'loose': 0} 100.0 %


In [13]:
i = 0
historic_pct = [] # historico donde iremos metiendo el porcentaje de cada interaccion
while True:
    i+=1 # para empezar desde 1
    score, data_X, data_y = play_and_learn(1000, debug=False)# 1000 interaciones
    pct = (score["win"]*100/(score["win"]+score["loose"])) # calculo de porcentajes
    historic_pct.append(pct) # lo agregamos al historico
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X): # si tenemos respuesta, si gano debe aprender
        model = model.partial_fit(data_X, data_y)

    
    if sum(historic_pct[-9:])==900: # si en el historic las ultimas 9 suman 900 para (100%)
        break

Iter: 1 - score: {'win': 552, 'loose': 448} 55.2 %
Iter: 2 - score: {'win': 559, 'loose': 441} 55.9 %
Iter: 3 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 4 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 5 - score: {'win': 311, 'loose': 689} 31.1 %
Iter: 6 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 7 - score: {'win': 311, 'loose': 689} 31.1 %
Iter: 8 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 9 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 10 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 11 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 12 - score: {'win': 309, 'loose': 691} 30.9 %
Iter: 13 - score: {'win': 315, 'loose': 685} 31.5 %
Iter: 14 - score: {'win': 348, 'loose': 652} 34.8 %
Iter: 15 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 16 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 17 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 18 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 19 - score: {'win': 356, 'loose': 644} 35.6 %
Iter: 20 - score: {'w

[Referencia](https://www.youtube.com/watch?v=eZpSGS7vF5Y)

In [14]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook,  export_png
output_notebook() # para hacer grafica de aprendizaje

Loading BokehJS ...

In [15]:
x = range(len(historic_pct))
print(x)
y = historic_pct
print(y)

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900) # 900 de ancho

p.line(x, y)
show(p)

range(0, 185)
[55.2, 55.9, 56.0, 33.7, 31.1, 34.6, 31.1, 34.1, 32.9, 32.7, 32.8, 30.9, 31.5, 34.8, 34.2, 33.8, 33.3, 33.9, 35.6, 33.2, 32.9, 33.4, 31.1, 34.3, 32.2, 34.6, 35.0, 33.1, 33.7, 31.7, 35.1, 32.3, 33.0, 33.0, 32.2, 33.2, 32.9, 33.4, 32.7, 32.1, 34.2, 32.8, 33.4, 34.4, 33.6, 34.3, 36.8, 33.2, 33.5, 32.9, 36.1, 31.9, 31.9, 34.8, 34.2, 31.7, 32.3, 34.0, 33.0, 33.2, 31.1, 32.1, 34.0, 33.8, 34.1, 32.8, 33.7, 33.3, 33.8, 33.4, 33.7, 33.5, 32.3, 33.2, 32.5, 33.2, 30.8, 30.8, 33.1, 32.2, 33.8, 32.4, 31.1, 32.6, 33.2, 34.4, 34.2, 31.2, 31.4, 33.9, 33.4, 32.9, 31.4, 35.5, 33.3, 33.8, 31.6, 31.0, 31.4, 31.8, 34.2, 33.5, 34.4, 33.5, 32.2, 32.5, 30.5, 34.5, 32.7, 32.5, 34.6, 35.4, 33.8, 36.2, 34.4, 33.2, 35.7, 34.6, 33.4, 34.3, 35.7, 33.5, 31.7, 32.6, 32.7, 33.0, 32.6, 32.7, 30.4, 33.1, 30.9, 32.1, 32.3, 31.0, 34.6, 34.2, 31.3, 33.5, 35.4, 35.5, 34.5, 34.9, 35.6, 32.2, 32.2, 33.6, 31.6, 34.7, 30.3, 55.4, 55.8, 57.2, 55.8, 56.1, 55.8, 56.2, 56.8, 52.9, 56.5, 56.2, 51.5, 59.1, 54.2, 57.0, 5

In [16]:
model.predict_proba([str_to_list("piedra")])


array([[0.00828141, 0.00912709, 0.97838791]])